# histooralcancer_dataloading

> How to load datasets for supervised learning, and SSL

Important: Note that have to download the data first from 

 

In [ ]:
#| default_exp histooralcancer_dataloading

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
import torch
from fastai.vision.all import *
# from self_supervised.augmentations import *
# from self_supervised.layers import *
from base_rbt.utils import *
import re
import os


Unzip example 

In [ ]:
#| hide

#!unzip -q -o "/content/drive/My Drive/histo_oral_cancer.zip" -d "/content/drive/My Drive/histo_oral_cancer"



Main dataloader functions:

In [ ]:
#| export

def label_func(x):
    # Function to extract the label from the parent directory name
    return x.parent.name

def get_supervised_histooralcancer_train_dls(bs, dataset_dir, size=256, device='cpu', pct_dataset=1.0, num_workers=12):
    
    train_dir = os.path.join(dataset_dir, "train")  # Corrected path to the train directory
    
    train_dir = os.path.join(dataset_dir, "train")  # Corrected path to the train directory

    # Paths to Normal and OSCC directories
    normal_dir = os.path.join(train_dir, "Normal")
    oscc_dir = os.path.join(train_dir, "OSCC")

    # Get image files from the Normal and OSCC directories
    fnames_normal = get_image_files(normal_dir)
    fnames_oscc = get_image_files(oscc_dir)

    # Sort filenames to ensure consistency
    fnames_normal = sorted(fnames_normal)
    fnames_oscc = sorted(fnames_oscc)

    # Apply subset size
    n_normal = int(len(fnames_normal) * (pct_dataset / 2))
    n_oscc = int(len(fnames_oscc) * (pct_dataset / 2))

    fnames_normal = fnames_normal[:n_normal]
    fnames_oscc = fnames_oscc[:n_oscc]

    # Combine the file lists
    fnames = fnames_normal + fnames_oscc

    # Data transformations

    # Create the DataLoader
    dls = ImageDataLoaders.from_path_func(
        path=train_dir,
        fnames=fnames,
        label_func=label_func,
        bs=bs,
        valid_pct=0.0,  # No validation split, using all for training
        device=device,
        num_workers=num_workers * (device == 'cuda')
    )

    return dls

def get_supervised_histooralcancer_test_dls(bs, dataset_dir, size=256, device='cpu', pct_dataset=1.0, num_workers=12):
    test_dir = os.path.join(dataset_dir, "test")  # Corrected path to the test directory
    val_dir = os.path.join(dataset_dir, "val")    # Path to the validation directory
    
    # Get image files from the testing and validation directories
    fnames = get_image_files(test_dir) + get_image_files(val_dir)

    # Apply subset size
    n = int(len(fnames) * pct_dataset)
    fnames = fnames[:n]

    # Data transformations
    # Create the DataLoader
    dls = ImageDataLoaders.from_path_func(
        path=test_dir,  # Path used for DataLoader
        fnames=fnames,
        label_func=label_func,
        bs=bs,
        valid_pct=0,  # No validation split for the test set
        device=device,
        drop_last=False,
        num_workers=num_workers * (device == 'cuda')
    )

    return dls



def get_bt_histooralcancer_train_dls(bs,size,device,pct_dataset=1.0,num_workers=12):

    pct_oscc = 0.025
    pct_normal = 1.0

    dataset_dir = "/content/drive/My Drive/histo_oral_cancer_resized"
    train_dir = os.path.join(dataset_dir, "train")  # Corrected path to the train directory

    # Paths to Normal and OSCC directories
    normal_dir = os.path.join(train_dir, "Normal")
    oscc_dir = os.path.join(train_dir, "OSCC")

    # Get image files from the Normal and OSCC directories
    fnames_normal = get_image_files(normal_dir)
    fnames_oscc = get_image_files(oscc_dir)

    # Sort filenames to ensure consistency
    fnames_normal = sorted(fnames_normal)
    fnames_oscc = sorted(fnames_oscc)

    # Apply subset size
    n_normal = int(len(fnames_normal) * pct_normal)
    n_oscc = int(len(fnames_oscc) * pct_oscc)

    fnames_normal = fnames_normal[:n_normal]
    fnames_oscc = fnames_oscc[:n_oscc]

    # Combine the file lists
    fnames = fnames_normal + fnames_oscc*10

    # Create the DataLoader
    dls = ImageDataLoaders.from_path_func(
        path=train_dir,
        fnames=fnames,
        label_func=label_func,
        bs=bs,
        valid_pct=0.0,  # No validation split, using all for training
        device=device,
        num_workers=num_workers * (device == 'cuda')
    )

    return dls

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()